In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import codecs
import os
import gzip
import sys
from bs4 import BeautifulSoup
from itertools import chain        
from contextlib import ExitStack 
import time
import json
from sklearn.metrics import silhouette_samples
from sklearn.metrics import adjusted_rand_score
from sklearn.cluster import KMeans
from multiprocessing.dummy import Pool as ThreadPool 
from multiprocessing.dummy import Lock as ThreadLock 
from multiprocessing.dummy import Value as ThreadValue
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import pairwise_distances
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import DBSCAN
from scipy.spatial.distance import cdist
from scipy.spatial.distance import cosine
from multiprocessing.dummy import Lock
from sklearn.feature_extraction.text import TfidfVectorizer, HashingVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import FastText
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models.fasttext import load_facebook_model
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV
from sklearn import model_selection
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold
from sklearn.base import BaseEstimator
from statistics import mean
from gensim.models import FastText
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neural_network import MLPClassifier
import json
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
import catboost
from sklearn.svm import SVC

In [4]:
vec = {}
with open('titles_stemming.json', 'r', encoding='utf8') as file:
    vec = json.load(file)
sentences = []
for value in vec.values():
    sentences.append(value)

In [6]:
model_fasttext = FastText('ft_freqprune_400K_100K_pq_300.bin')

In [7]:
with open('texts_embeddings.json', 'r', encoding='utf8') as file:
    texts_embedding_from_json = json.load(file)

In [8]:
for key, val in texts_embedding_from_json.items():
    embedding = np.array(list(map(float, texts_embedding_from_json[key])))
    texts_embedding_from_json[key] = embedding

In [9]:
class Import:
    def __init__(self):
        self.doc_to_title = {}
        #self.model_fasttext = FastText('/Users/egor/Desktop/Техносфера/MachineLearning/project/ft_freqprune_400K_100K_pq_300.bin')
        with open('docs_titles.tsv') as f:
            for num_line, line in enumerate(f):
                if num_line == 0:
                    continue
                data = line.strip().split('\t', 1)
                doc_id = int(data[0])
                if len(data) == 1:
                    title = ''
                else:
                    title = data[1]
                self.doc_to_title[doc_id] = title
        self.doc_to_vec = TfidfVectorizer().fit_transform([self.doc_to_title[i] for i in self.doc_to_title.keys()])
        self.top_k_cosine_titles = 25 # k ближайших косинусных расстояний для заголовков
        self.top_k_cosine_texts = 3 # k ближайших косинусных расстояний для заголовков
    
    def make_train(self):
        train_data = pd.read_csv('train_groups.csv')
        traingroups_titledata = {}
        for i in range(len(train_data)):
            new_doc = train_data.iloc[i]
            doc_group = new_doc['group_id']
            doc_id = new_doc['doc_id']
            target = new_doc['target']
            title = self.doc_to_title[doc_id]
            if doc_group not in traingroups_titledata:
                traingroups_titledata[doc_group] = []
            traingroups_titledata[doc_group].append((doc_id, title, target))
        
        y_train = []
        X_train = []
        groups_train = []
        for new_group in traingroups_titledata:
            docs = traingroups_titledata[new_group]
            for k, (doc_id, title, target_id) in enumerate(docs):
                y_train.append(target_id)
                groups_train.append(new_group)
                all_dist = []
                words = set(title.strip().split())
                for j in range(0, len(docs)):
                    if k == j:
                        continue
                    doc_id_j, title_j, target_j = docs[j]
                    words_j = set(title_j.strip().split())
                    all_dist.append(len(words.intersection(words_j)))
                X_train.append(sorted(all_dist, reverse=True)[0:25]    )
        X_train = np.array(X_train)
        y_train = np.array(y_train)
        groups_train = np.array(groups_train)
        
        # Класстеризация внутри группы:
        i = 0
        My_X = np.zeros((train_data.shape[0], self.top_k_cosine_titles + 2)) # кол-во новых фичей
        groups = train_data.groupby('group_id')
        rand_index = []
        for group_id, group_indx in groups.groups.items():
            group = train_data.iloc[group_indx]
            docs_id = group.doc_id.values
            # сначала заберем все файлы из группы
            embedded_texts = []
            group_titles = []
            for idx in docs_id:
                group_titles.append(vec[str(idx)])
                embedded_texts.append(texts_embedding_from_json[str(idx)])
            embedded_titles = model_fasttext.wv[group_titles]
            targets = group.target.values
            model_titles = DBSCAN(0.6, metric="cosine", min_samples=8)
            labels_titles = model_titles.fit_predict(embedded_titles)
            if np.unique(labels_titles).size > 1:
                silhouette = silhouette_samples(embedded_titles, labels_titles)
            else:
                silhouette = np.ones(len(labels_titles)) # если всего один кластер
            model_texts = DBSCAN(0.1, metric="cosine", min_samples=5)
            labels_texts = model_texts.fit_predict(embedded_texts)
            # добавим фичи из расстояний
            # получим матрицу из попарных расстояний
            cos_distances = cosine_similarity(embedded_titles)
            sorted_dist = np.apply_along_axis(np.sort, axis=1, arr=cos_distances)[:, ::-1]
            cos_features = sorted_dist[:, 1: self.top_k_cosine_titles + 1]
            
            cos_distances_texts = cosine_similarity(embedded_texts)
            sorted_dist_texts = np.apply_along_axis(np.sort, axis=1, arr=cos_distances_texts)[:, ::-1]
            cos_features_texts = sorted_dist_texts[:, 1: self.top_k_cosine_texts + 1]
            
            
            My_X[i: i + group_indx.size, :self.top_k_cosine_titles] = cos_features
            My_X[i: i + group_indx.size, -2] = labels_titles
            My_X[i: i + group_indx.size, -1] = labels_texts
            i += group_indx.size
        
        
        X_train = np.concatenate((X_train, My_X), axis=1)
        return X_train, y_train, groups_train
    
    def make_test(self):
        test_data = pd.read_csv('test_groups.csv')
        testgroups_titledata = {}
        for i in range(len(test_data)):
            new_doc = test_data.iloc[i]
            doc_group = new_doc['group_id']
            doc_id = new_doc['doc_id']
            title = self.doc_to_title[doc_id]
            if doc_group not in testgroups_titledata:
                testgroups_titledata[doc_group] = []
            testgroups_titledata[doc_group].append((doc_id, title))
            
        X_test = []
        groups_test = []
        for new_group in testgroups_titledata:
            docs = testgroups_titledata[new_group]
            for k, (doc_id, title) in enumerate(docs):
                groups_test.append(new_group)
                all_dist = []
                words = set(title.strip().split())
                for j in range(0, len(docs)):
                    if k == j:
                        continue
                    doc_id_j, title_j = docs[j]
                    words_j = set(title_j.strip().split())
                    all_dist.append(len(words.intersection(words_j)))
                X_test.append(sorted(all_dist, reverse=True)[0:25])
        X_test = np.array(X_test)
        
        i = 0
        My_X = np.zeros((test_data.shape[0], self.top_k_cosine_titles + 2)) # кол-во новых фичей
        groups = test_data.groupby('group_id')
        for group_id, group_indx in groups.groups.items():
            group = test_data.iloc[group_indx]
            docs_id = group.doc_id.values
            group_titles = []
            embedded_texts = []
            for idx in docs_id:
                group_titles.append(vec[str(idx)])
                embedded_texts.append(texts_embedding_from_json[str(idx)])
            embedded_titles = model_fasttext.wv[group_titles]
            model_titles = DBSCAN(0.6, metric="cosine", min_samples=8)
            labels_titles = model_titles.fit_predict(embedded_titles)
            if np.unique(labels_titles).size > 1:
                silhouette = silhouette_samples(embedded_titles, labels_titles)
            else:
                silhouette = np.ones(len(labels_titles)) # если всего один кластер
            model_texts = DBSCAN(0.1, metric="cosine", min_samples=5)
            labels_texts = model_texts.fit_predict(embedded_texts)
            cos_distances = cosine_similarity(embedded_titles)
            sorted_dist = np.apply_along_axis(np.sort, axis=1, arr=cos_distances)[:, ::-1]
            cos_features = sorted_dist[:, 1: self.top_k_cosine_titles + 1]
            
            cos_distances_texts = cosine_similarity(embedded_texts)
            sorted_dist_texts = np.apply_along_axis(np.sort, axis=1, arr=cos_distances_texts)[:, ::-1]
            cos_features_texts = sorted_dist_texts[:, 1: self.top_k_cosine_texts + 1]
            
            
            My_X[i: i + group_indx.size, :self.top_k_cosine_titles] = cos_features
            My_X[i: i + group_indx.size, -2] = labels_titles
            My_X[i: i + group_indx.size, -1] = labels_texts
            i += group_indx.size
            
            #My_X[i: i + group_indx.size, self.top_k_cosine_titles:self.top_k_cosine_titles
            #     + self.top_k_cosine_texts] = cos_features_texts
            # валидация дала DBSCAN(0.6, metric="cosine", min_samples=8)
        
        X_test = np.concatenate((X_test, My_X), axis=1)
        groups_test = np.array(groups_test)
        return X_test, groups_test, test_data['pair_id'].tolist()

In [10]:
importer = Import()

In [11]:
X_train, y_train, groups_train = importer.make_train()

In [12]:
X_test, groups_test, pair_ids = importer.make_test()

In [14]:
class Voiting(BaseEstimator):
    def __init__(self):
        self.clf_e_1 = RandomForestClassifier(max_depth=10)
        self.clf_e_2 = ExtraTreesClassifier(max_depth=15)
        self.clf_e_3 = LogisticRegression()
        self.clf_e_4 = catboost.CatBoostClassifier()
        self.clf_e_5 = MLPClassifier(max_iter=800, hidden_layer_sizes=(10, 10), random_state=42)
        self.clf_e_6 = KNeighborsClassifier(10)

    def fit(self, X, y):
        self.clf_e_1.fit(X, y)
        self.clf_e_2.fit(X, y)
        self.clf_e_3.fit(X, y)
        self.clf_e_4.fit(X, y)
        self.clf_e_5.fit(X, y)
        self.clf_e_6.fit(X, y)

    def predict(self, X):
        self.pred = np.transpose(np.array([self.clf_e_1.predict(X),
                                self.clf_e_2.predict(X),
                                self.clf_e_3.predict(X),
                                self.clf_e_4.predict(X),
                                self.clf_e_5.predict(X),
                                self.clf_e_6.predict(X)]))
        def pr(line):
            ar, count = np.unique(line, return_counts = True)
            return ar[np.argmax(count)]
        predict = np.array([pr(line) for line in self.pred])
        return predict   

In [15]:
my_sk = StandardScaler()
X_train_sk = my_sk.fit_transform(X_train)
X_test_sk = my_sk.fit_transform(X_test)

In [78]:
kfold = StratifiedKFold(n_splits=5)

In [18]:
DTC = DecisionTreeClassifier()

adaDTC = AdaBoostClassifier(DTC, random_state=7)

ada_param_grid = {"base_estimator__criterion" : ["gini", "entropy"],
              "base_estimator__splitter" :   ["best", "random"],
              "algorithm" : ["SAMME","SAMME.R"],
              "n_estimators" :[1,2],
              "learning_rate":  [0.0001, 0.001, 0.01, 0.1, 0.2, 0.3,1.5]}

gsadaDTC = GridSearchCV(adaDTC,param_grid = ada_param_grid, cv=kfold, scoring="f1", n_jobs= 4, verbose = 1)

gsadaDTC.fit(X_train_sk,y_train)

ada_best = gsadaDTC.best_estimator_

Fitting 5 folds for each of 112 candidates, totalling 560 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:   16.7s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:  1.1min
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:  1.9min
[Parallel(n_jobs=4)]: Done 560 out of 560 | elapsed:  2.3min finished


In [19]:
res_ada = ada_best.predict_proba(X_train_sk)[:, 1]
for th in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]:
    print(f1_score(y_train, np.array(res_ada > th, dtype=int)))

0.4466148428675835
0.4466148428675835
0.9991082045184304
0.9991082045184304
0.9991082045184304
0.9991066110780226
0.9991066110780226


In [20]:
ExtC = ExtraTreesClassifier()


## Search grid for optimal parameters
ex_param_grid = {"max_depth": [None],
              "max_features": [1, 3, 10],
              "min_samples_split": [2, 3, 10],
              "min_samples_leaf": [1, 3, 10],
              "bootstrap": [False],
              "n_estimators" :[100,300],
              "criterion": ["gini"]}


gsExtC = GridSearchCV(ExtC,param_grid = ex_param_grid, cv=kfold, scoring="f1", n_jobs= 4, verbose = 1)

gsExtC.fit(X_train_sk,y_train)

ExtC_best = gsExtC.best_estimator_

# Best score
gsExtC.best_score_

Fitting 5 folds for each of 54 candidates, totalling 270 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:   30.4s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:  2.0min
[Parallel(n_jobs=4)]: Done 270 out of 270 | elapsed:  3.9min finished


0.6886489405661631

In [21]:
res_extra = ExtC_best.predict_proba(X_train_sk)[:, 1]
for th in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]:
    print(f1_score(y_train, np.array(res_extra > th, dtype=int)))

0.6777193691872221
0.7444523979957051
0.7746031746031746
0.7861827721906428
0.7651951375559822
0.7242769500438212
0.6437984496124032


In [22]:
# RFC Parameters tunning 
RFC = RandomForestClassifier()


## Search grid for optimal parameters
rf_param_grid = {"max_depth": [None],
              "max_features": [1, 3, 10],
              "min_samples_split": [2, 3, 10],
              "min_samples_leaf": [1, 3, 10],
              "bootstrap": [False],
              "n_estimators" :[100,300],
              "criterion": ["gini"]}


gsRFC = GridSearchCV(RFC,param_grid = rf_param_grid, cv=kfold, scoring="f1", n_jobs= 4, verbose = 1)

gsRFC.fit(X_train_sk,y_train)

RFC_best = gsRFC.best_estimator_

# Best score
gsRFC.best_score_

Fitting 5 folds for each of 54 candidates, totalling 270 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:   54.6s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:  7.2min
[Parallel(n_jobs=4)]: Done 270 out of 270 | elapsed: 16.1min finished


0.6930063619939852

In [23]:
res_forest = RFC_best.predict_proba(X_train_sk)[:, 1]
for th in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]:
    print(f1_score(y_train, np.array(res_forest > th, dtype=int)))

0.7154108131119625
0.8211611636185099
0.8616675690308608
0.8792143066549399
0.8671065032987747
0.8349547131834955
0.7633977397010573


In [25]:
# Gradient boosting tunning

GBC = GradientBoostingClassifier()
gb_param_grid = {'loss' : ["deviance"],
              'n_estimators' : [100,200,300],
              'learning_rate': [0.1, 0.05, 0.01],
              'max_depth': [4, 8],
              'min_samples_leaf': [100,150],
              'max_features': [0.3, 0.1] 
              }

gsGBC = GridSearchCV(GBC,param_grid = gb_param_grid, cv=kfold, scoring="f1", n_jobs= 4, verbose = 1)

gsGBC.fit(X_train_sk,y_train)

GBC_best = gsGBC.best_estimator_

# Best score
gsGBC.best_score_

Fitting 5 folds for each of 72 candidates, totalling 360 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:  1.8min
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:  9.1min
[Parallel(n_jobs=4)]: Done 360 out of 360 | elapsed: 19.2min finished


0.6996043329287585

In [47]:
res_grad = GBC_best.predict_proba(X_train_sk)[:, 1]
for th in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]:
    print(f1_score(y_train, np.array(res_grad > th, dtype=int)))

0.6474201474201473
0.7179239589619794
0.7430332261521972
0.7449498619386717
0.7347970304849154
0.7008635578583765
0.6264921062764729


In [67]:
# Gradient boosting tunning

XGB = xgb.XGBClassifier()
xgb_param_grid = {'alpha' : [0.01, 0.1],
              'n_estimators' : [10,20,30],
              'eval_metric': ['auc'],
              'max_depth': [4, 6],
              'scale_pos_weight': [1, 2, 2.4] 
              }

gsXGB = GridSearchCV(XGB,param_grid = xgb_param_grid, cv=kfold, scoring="f1", n_jobs= 4, verbose = 1)

gsXGB.fit(X_train_sk,y_train)

XGB_best = gsXGB.best_estimator_

# Best score
gsXGB.best_score_

Fitting 5 folds for each of 36 candidates, totalling 180 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:   18.8s
[Parallel(n_jobs=4)]: Done 180 out of 180 | elapsed:  1.5min finished


0.7130615786232939

In [68]:
res_xgb = XGB_best.predict_proba(X_train_sk)[:, 1]
for th in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]:
    print(f1_score(y_train, np.array(res_xgb > th, dtype=int)))

0.5776690546915954
0.6690579175018462
0.7080266386260077
0.7354214413037942
0.7475192943770672
0.7447723565178704
0.7171717171717172


In [54]:
from sklearn.ensemble import StackingClassifier

In [69]:
estimators = [('rf', RFC_best), ('et', ExtC_best), ('xgb', XGB_best), ('gb', GBC_best)]

Вот здесь я крутил модели и потом делал стакинг

In [70]:
stack = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression())

In [71]:
stack.fit(X_train_sk,y_train)

StackingClassifier(estimators=[('rf',
                                RandomForestClassifier(bootstrap=False,
                                                       max_features=10,
                                                       min_samples_leaf=10,
                                                       min_samples_split=3)),
                               ('et',
                                ExtraTreesClassifier(max_features=10,
                                                     min_samples_leaf=10,
                                                     min_samples_split=3)),
                               ('xgb',
                                XGBClassifier(alpha=0.1, base_score=0.5,
                                              booster='gbtree',
                                              colsample_bylevel=1,
                                              colsample_bynode=1,
                                              colsample_bytree=1,
                               

In [72]:
stack_res = stack.predict_proba(X_train_sk)[:, 1]
for th in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]:
    print(f1_score(y_train, np.array(stack_res > th, dtype=int)))

0.6769297017978254
0.7373377449732756
0.7503109023075861
0.7529481132075471
0.7352987545325557
0.7127605490594814
0.6491522265697782


Здесь ансамбль над стекингом (последний скор без этого был)

In [81]:
from sklearn.ensemble import VotingClassifier

In [82]:
votingC = VotingClassifier(estimators=[('rfc', RFC_best), ('extc', ExtC_best), ('stack', stack), ('xgb', XGB_best)], voting='soft', n_jobs=4)

votingC = votingC.fit(X_train_sk, y_train)

In [83]:
vote = votingC.predict_proba(X_train_sk)[:, 1]
for th in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]:
    print(f1_score(y_train, np.array(vote > th, dtype=int)))

0.661152141802068
0.7429577464788732
0.7737491877842755
0.7898161244695899
0.7842418689876316
0.765865063460254
0.6934414148857775


ансамбль над стекингом

In [84]:
y_pls = np.array(votingC.predict_proba(X_test_sk)[:, 1] > 0.4, dtype=int)

просто стекинг (у меня еще пороги разные получались иногда)

In [73]:
y_pls = np.array(stack.predict_proba(X_test_sk)[:, 1] > 0.4, dtype=int)

In [85]:
with open("subpobeda.csv", "w", encoding='utf-8') as outfile:
    outfile.write(str("pair_id,target\n"))
    for i in range(len(y_pls)):
        outfile.write(str(pair_ids[i]) + str(",") + str(y_pls[i]) + str("\n"))